In [ ]:
# Callback period jitter
#
# Get trace data using the provided launch file:
#    $ ros2 launch pendulum_bringup pendulum_bringup.launch.py trace:=True
#    (wait a few seconds, then kill with Ctrl+C)
#
# (optional) convert trace data:
#    $ ros2 run tracetools_analysis convert ~/.ros/tracing/pendulum/ust -o pendulum_converted
#
# OR
#
# Use the provided sample converted trace file, changing the path below to:
#    'sample_data/converted_pendulum'

In [ ]:
#path = '~/.ros/tracing/pendulum/ust'
path = 'sample_data/pendulum_converted'

In [ ]:
import sys
import datetime as dt

from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
from bokeh.models import PrintfTickFormatter
import numpy as np
import pandas as pd

from tracetools_analysis.loading import load_file
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.utils.ros2 import Ros2DataModelUtil

In [ ]:
# Process
events = load_file(path)
handler = Ros2Handler.process(events)

In [ ]:
data_util = Ros2DataModelUtil(handler.data)

callback_symbols = data_util.get_callback_symbols()
# select timer and subscription callbacks
selected_callbacks = {key: value for (key, value) in callback_symbols.items() if 'pendulum::' in value }

output_notebook()
psize = 450
colours = ['#29788E', '#DD4968', '#410967', '#D19275']
legends = ['pendulum_driver - Timer cb',
           'pendulum_driver - Subscription cb',
           'pendulum_controller - Timer cb',
           'pendulum_controller - Subscription cb']

In [ ]:
# Plot callback period jitter separately
plot_i = 0
for obj, symbol in selected_callbacks.items():
    owner_info = data_util.get_callback_owner_info(obj)
    if owner_info is None:
        owner_info = '[unknown]'

    # Delta time
    df = data_util.get_callback_durations(obj)
    # Convert to delta time in ms
    time_diff = df['timestamp'] - df['timestamp'].shift()
    # Remove first row cause is NaN
    df = df.iloc[1:]  
    time_diff = time_diff.iloc[1:] 
    df['time_delta'] = time_diff.astype(int)/ 10**6
    df['iterations'] = df.index
    source = ColumnDataSource(df)
    
    deltaTime = figure(
        title=owner_info,
        x_axis_label=f'Iterations',
        y_axis_label='Delta Time (ms)',
        plot_width=psize, plot_height=psize,
    )
    deltaTime.title.align = 'center'
    deltaTime.line(
        x='iterations',
        y='time_delta',
        legend_label=legends[plot_i],
        line_width=2,
        source=source,
        line_color=colours[plot_i],
    )
    deltaTime.legend.label_text_font_size = '11px'

    # Histogram
    del_hist, edges = np.histogram(df['time_delta'], bins='auto')
    delta_hist = pd.DataFrame({
        'time_delta': del_hist, 
        'left': edges[:-1], 
        'right': edges[1:],
    })
    hist = figure(
        title='Callback period histogram',
        x_axis_label='Delta Time (ms)',
        y_axis_label='frequency',
        plot_width=psize, plot_height=psize,
    )
    hist.title.align = 'center'
    hist.quad(
        bottom=0,
        top=delta_hist['time_delta'], 
        left=delta_hist['left'],
        right=delta_hist['right'],
        fill_color=colours[plot_i],
        line_color=colours[plot_i],
    )
    
    plot_i += 1
    show(row(deltaTime, hist))

In [ ]:
# Plot period jitter in one plot
earliest_date = None
for obj, symbol in selected_callbacks.items():
    df = data_util.get_callback_durations(obj)
    thedate = df.loc[:, 'timestamp'].iloc[0]
    if earliest_date is None or thedate <= earliest_date:
        earliest_date = thedate

duration = figure(
    title='Callback period',
    x_axis_label=f'Iterations',
    y_axis_label='Delta Time (ms)',
    plot_width=psize, plot_height=psize,
)

plot_i = 0
for obj, symbol in selected_callbacks.items():
    df = data_util.get_callback_durations(obj)
    # Convert to delta time in ms
    time_diff = df['timestamp'] - df['timestamp'].shift()
    # Remove first row cause is NaN
    df = df.iloc[1:]  
    time_diff = time_diff.iloc[1:] 
    df['time_delta'] = time_diff.astype(int)/ 10**6
    df['iterations'] = df.index
    source = ColumnDataSource(df)
    deltaTime.title.align = 'center'
    deltaTime.line(
        x='iterations',
        y='time_delta',
        legend_label=legends[plot_i],
        line_width=2,
        source=source,
        line_color=colours[plot_i],
    )
    deltaTime.legend.label_text_font_size = '11px'
    
    # Print statistics
    deltaTime_min = np.min(df['time_delta'])
    deltaTime_max = np.max(df['time_delta'])
    deltaTime_mean = np.mean(df['time_delta'])
    deltaTime_std = np.std(df['time_delta'])
    
    print('callback: ', legends[plot_i])
    print("min:  {} ms".format(round(deltaTime_min,4)))
    print("max:  {} ms".format(round(deltaTime_max,4)))
    print("mean: {} ms".format(round(deltaTime_mean,4)))
    print("std:  {} ms".format(round(deltaTime_std,4)))

    plot_i += 1
   
    
show(deltaTime)